In [29]:
import sqlite3
import pandas as pd

# Loading CSV files into Pandas DataFrames

products = pd.read_csv('/Users/nani/Desktop/Fetch/PRODUCTS_TAKEHOME.csv')
transactions = pd.read_csv('/Users/nani/Desktop/Fetch/TRANSACTION_TAKEHOME.csv')
users = pd.read_csv('/Users/nani/Desktop/Fetch/USER_TAKEHOME.csv')
conn = sqlite3.connect(':memory:')

#DataFrame into a SQL table (table names are lowercase)
products.to_sql('products', conn, index=False, if_exists='replace')
transactions.to_sql('transactions', conn, index=False, if_exists='replace')
users.to_sql('users', conn, index=False, if_exists='replace')

100000

In [30]:
#SQL queries
#Closed-ended questions:

In [31]:
#What are the top 5 brands by receipts scanned among users 21 and over?

In [32]:
query_alt = """
WITH eligible_users AS (
    SELECT 
        ID AS USER_ID,
        BIRTH_DATE,
        (julianday('now') - julianday(BIRTH_DATE)) / 365.25 AS AGE
    FROM users
    WHERE BIRTH_DATE IS NOT NULL
),
valid_transactions AS (
    SELECT 
        t.RECEIPT_ID, 
        t.USER_ID, 
        t.BARCODE
    FROM transactions t
    WHERE t.RECEIPT_ID IS NOT NULL
),
joined_data AS (
    SELECT 
        vt.RECEIPT_ID,
        p.BRAND
    FROM valid_transactions vt
    JOIN eligible_users eu ON vt.USER_ID = eu.USER_ID
    JOIN products p ON vt.BARCODE = p.BARCODE
    WHERE eu.AGE >= 21 AND p.BRAND IS NOT NULL
)
SELECT 
    BRAND, 
    COUNT(RECEIPT_ID) AS receipts_scanned
FROM joined_data
GROUP BY BRAND
ORDER BY receipts_scanned DESC
LIMIT 5;
"""


#result
print("Top 5 Brands by Receipts Scanned Among Users 21 and Over (without any null values):")
print(df_alt_query)


Top 5 Brands by Receipts Scanned Among Users 21 and Over (without any null values):
             BRAND  receipts_scanned
0      NERDS CANDY                 6
1             DOVE                 6
2          TRIDENT                 4
3  SOUR PATCH KIDS                 4
4           MEIJER                 4


In [33]:
#What are the top 5 brands by sales among users that have had their account for at least six months?

In [34]:
query_top_sales_6months = """
WITH eligible_users AS (
    SELECT 
        ID AS USER_ID,
        CREATED_DATE,
        (julianday('now') - julianday(CREATED_DATE)) / 30.44 AS account_age_months
    FROM users
    WHERE CREATED_DATE IS NOT NULL
),
valid_transactions AS (
    SELECT 
        t.USER_ID,
        t.BARCODE,
        t.FINAL_SALE
    FROM transactions t
    WHERE t.FINAL_SALE IS NOT NULL
),
joined_data AS (
    SELECT 
        vt.FINAL_SALE,
        p.BRAND
    FROM valid_transactions vt
    JOIN eligible_users eu ON vt.USER_ID = eu.USER_ID
    JOIN products p ON vt.BARCODE = p.BARCODE
    WHERE eu.account_age_months >= 6 AND p.BRAND IS NOT NULL
)
SELECT 
    BRAND,
    ROUND(SUM(FINAL_SALE), 2) AS total_sales
FROM joined_data
GROUP BY BRAND
ORDER BY total_sales DESC
LIMIT 5;
"""

df_top_sales_6months = pd.read_sql(query_top_sales_6months, conn)
print("Top 5 Brands by Sales Among Users with Accounts Older Than 6 Months:")
print(df_top_sales_6months)


Top 5 Brands by Sales Among Users with Accounts Older Than 6 Months:
         BRAND  total_sales
0          CVS        72.00
1      TRIDENT        46.72
2         DOVE        42.88
3  COORS LIGHT        34.96
4       QUAKER        16.60


In [35]:
#What is the percentage of sales in the Health & Wellness category by generation?

In [36]:
query_health_sales_by_generation = """
WITH user_generations AS (
    SELECT 
        ID AS USER_ID,
        BIRTH_DATE,
        CASE
            WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) >= 1997 THEN 'Gen Z'
            WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1981 AND 1996 THEN 'Millennials'
            WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1965 AND 1980 THEN 'Gen X'
            WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1946 AND 1964 THEN 'Boomers'
            ELSE 'Silent Generation'
        END AS generation
    FROM users
    WHERE BIRTH_DATE IS NOT NULL
),
joined_data AS (
    SELECT 
        ug.generation,
        t.FINAL_SALE,
        p.CATEGORY_1
    FROM transactions t
    JOIN user_generations ug ON t.USER_ID = ug.USER_ID
    JOIN products p ON t.BARCODE = p.BARCODE
    WHERE t.FINAL_SALE IS NOT NULL
)
SELECT 
    generation,
    ROUND(SUM(CASE WHEN CATEGORY_1 = 'Health & Wellness' THEN FINAL_SALE ELSE 0 END), 2) AS health_sales,
    ROUND(SUM(FINAL_SALE), 2) AS total_sales,
    ROUND(SUM(CASE WHEN CATEGORY_1 = 'Health & Wellness' THEN FINAL_SALE ELSE 0 END) * 100.0 / SUM(FINAL_SALE), 2) AS health_sales_percentage
FROM joined_data
GROUP BY generation
ORDER BY health_sales_percentage DESC;
"""

df_health_sales_by_generation = pd.read_sql(query_health_sales_by_generation, conn)
print("Percentage of Sales in Health & Wellness Category by Generation:")
print(df_health_sales_by_generation)


Percentage of Sales in Health & Wellness Category by Generation:
          generation  health_sales  total_sales  health_sales_percentage
0        Millennials         59.13       147.97                    39.96
1            Boomers         89.03       225.57                    39.47
2              Gen X         41.50       138.95                    29.87
3  Silent Generation          0.00        13.42                     0.00
4              Gen Z          0.00         1.97                     0.00


In [37]:
#Open-ended questions: for these, make assumptions and clearly state them when answering the question.

In [38]:
#Who are Fetch’s power users?

In [39]:
# A power user is a person or a record that has scanned a high number of receipts or a record that has high purchase value

# For example calculating top 5%  of all users in total FINAL_SALE 

In [55]:
import sqlite3
import pandas as pd

# Load CSV files 

products = pd.read_csv('/Users/nani/Desktop/Fetch/PRODUCTS_TAKEHOME.csv')
transactions = pd.read_csv('/Users/nani/Desktop/Fetch/TRANSACTION_TAKEHOME.csv')
users = pd.read_csv('/Users/nani/Desktop/Fetch/USER_TAKEHOME.csv')
conn = sqlite3.connect(':memory:')
products.to_sql('products', conn, index=False, if_exists='replace')
transactions.to_sql('transactions', conn, index=False, if_exists='replace')
users.to_sql('users', conn, index=False, if_exists='replace')
transactions['FINAL_SALE'] = pd.to_numeric(transactions['FINAL_SALE'], errors='coerce')
cleaned_transactions = transactions.dropna(subset=['FINAL_SALE', 'RECEIPT_ID'])

# Aggregating  user-level totals
user_sales = (
    cleaned_transactions
    .groupby('USER_ID')
    .agg(total_receipts=('RECEIPT_ID', 'nunique'),
         total_spend=('FINAL_SALE', 'sum'))
    .reset_index()
)

# Calculating 95th percentiles
receipt_threshold = user_sales['total_receipts'].quantile(0.95)
spend_threshold = user_sales['total_spend'].quantile(0.95)

# Calculating  power users
power_users = user_sales[
    (user_sales['total_receipts'] >= receipt_threshold) |
    (user_sales['total_spend'] >= spend_threshold)
].sort_values(by='total_spend', ascending=False)

# result
print("Top Power Users (top 5% by spend or receipts):")
print(power_users.head(10))

Top Power Users (top 5% by spend or receipts):
                        USER_ID  total_receipts  total_spend
10392  630789e1101ae272a4852287               1       925.64
12195  63af23db9f3fc9c7546fdbec               1       476.34
14830  650874eafe41d365c2ee11d2               1       267.29
13636  645add3bffe0d7e043ef1b63               4       227.93
11564  637257e75fdbb03aa198a310               4       194.14
12321  63ba250039c79dcbdd4fbc6c               1       187.04
15101  6535138fdda97e6b1342ea7f               3       146.65
6109   610b3d5002b8c34bbf3a589d               2       144.96
1527   5d61b8e71ddc4058bd98f776               1       143.94
2248   5e022fe1daccde120544eb42               3       140.74
